# Install and Import Libraries
Code to install the required libraries (diffusers, torch, torchvision, matplotlib) and import them.

In [ ]:
# Install required libraries
!pip install diffusers torch torchvision matplotlib

# Import necessary libraries
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from diffusers import DDPMScheduler, UNet2DModel
import matplotlib.pyplot as plt
import os

# Prepare Dataset
Define transformations and create a dataset using ImageFolder with the specified dataroot.

In [ ]:
# Define data transformations
transform=transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Set the dataset root directory
dataroot = "data/wiki"

# Create the dataset
dataset = ImageFolder(root=dataroot, transform=transform)

# Check the number of samples in the dataset
print(f"Number of images in the dataset: {len(dataset)}")

# Setup Data Loader
Initialize the DataLoader with parameters like batch_size and shuffle for batching the dataset.

In [ ]:
from torch.utils.data import DataLoader

# Initialize DataLoader
batch_size = 16  # Define batch size
shuffle = True   # Shuffle the dataset for training

# Create DataLoader for batching
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Check the number of batches
num_batches = len(dataloader)
print(f"Number of batches: {num_batches}")

# Define Diffusion Model and Trainer
Instantiate the UNet2DModel, set up the noise scheduler, and define the optimizer and initial training configuration.

In [ ]:
# Define the diffusion model
model = UNet2DModel(
    sample_size=64,  # Image size
    in_channels=3,   # Number of input channels (RGB)
    out_channels=3,  # Number of output channels (RGB)
    layers_per_block=2,
    block_out_channels=(128, 256, 512, 512),
    down_block_types=(
        "DownBlock2D", "DownBlock2D", "DownBlock2D", "AttnDownBlock2D"
    ),
    up_block_types=(
        "AttnUpBlock2D", "UpBlock2D", "UpBlock2D", "UpBlock2D"
    )
)

# Define the noise scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training parameters
num_epochs = 15  # Number of epochs

# Training Loop
Implement the loop to iterate over data batches, add noise, predict noise, and update model weights. Note: This section appears twice in the notebook, but only one implementation is required.

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for step, (images, _) in enumerate(dataloader):
        # Move images to the device
        images = images.to(device)

        # Sample noise
        noise = torch.randn_like(images).to(device)

        # Sample random timesteps
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (images.shape[0],), device=device).long()

        # Add noise to the images
        noisy_images = noise_scheduler.add_noise(images, noise, timesteps)

        # Predict the noise
        noise_pred = model(noisy_images, timesteps).sample

        # Compute loss (mean squared error)
        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss every 100 steps
        if step % 100 == 0:
            print(f"Step {step}/{len(dataloader)}, Loss: {loss.item()}")

# Generate and Display Images
After training, set the model to evaluation mode, generate images using the reverse diffusion process, and display them using matplotlib.

In [ ]:
# Generate and display images after training using the correct de-noising loop
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    num_images = 16  # Total images to generate
    rows, cols = 4, 4  # 4x4 grid
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = axes.flatten()  # Flatten the array to iterate easily
    
    for i in range(num_images):
        # Start from random noise with the same size as training images (64x64 with 3 channels)
        noisy_image = torch.randn(1, 3, 64, 64).to(device)
        
        # Reverse diffusion process
        for t in reversed(range(noise_scheduler.num_train_timesteps)):
            # Get noise prediction from the model
            noise_pred = model(noisy_image, t).sample  
            # Perform a de-noising step using the predicted noise
            step_output = noise_scheduler.step(noise_pred, t, noisy_image)
            noisy_image = step_output.prev_sample
        
        # Denormalize and prepare image for display
        generated_image = (noisy_image.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5).clip(0, 1)
        axes[i].imshow(generated_image)
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()

In [ ]:
from scipy import linalg
import torch.nn as nn
from torchvision.models import inception_v3
from torchvision.models.inception import Inception_V3_Weights

class InceptionV3FeatureExtractor:
    def __init__(self, device='cpu'):
        self.device = device
        weights = Inception_V3_Weights.DEFAULT
        self.inception = inception_v3(weights=weights)
        self.inception.eval()
        self.inception.fc = nn.Identity()
        self.inception.to(device)
        self.preprocess = weights.transforms()

    def extract_features(self, images):
        features = []
        with torch.no_grad():
            for img in images:
                # If img is a tensor of shape (3, H, W), convert it to PIL image via the official transforms
                inp = self.preprocess(img).unsqueeze(0).to(self.device)
                feature = self.inception(inp)
                features.append(feature.cpu().numpy())
        return np.concatenate(features, axis=0)

def calculate_fid(real_features, fake_features):
    mu_real = np.mean(real_features, axis=0)
    sigma_real = np.cov(real_features, rowvar=False)
    mu_fake = np.mean(fake_features, axis=0)
    sigma_fake = np.cov(fake_features, rowvar=False)
    mean_diff_squared = np.sum((mu_real - mu_fake) ** 2)
    covmean = linalg.sqrtm(sigma_real.dot(sigma_fake))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    trace_term = np.trace(sigma_real + sigma_fake - 2 * covmean)
    fid = mean_diff_squared + trace_term
    return fid

def generate_fake_image(model, noise_scheduler, device):
    with torch.no_grad():
        # Start from random noise (1, 3, 64, 64)
        noisy_image = torch.randn(1, 3, 64, 64).to(device)
        # Reverse diffusion process
        for t in reversed(range(noise_scheduler.num_train_timesteps)):
            noise_pred = model(noisy_image, t).sample
            step_output = noise_scheduler.step(noise_pred, t, noisy_image)
            noisy_image = step_output.prev_sample
    # Output image in same scale as training images (assumed normalized to [-1, 1])
    return noisy_image.squeeze(0)

def compute_fid(real_imgs, model, noise_scheduler, device, num_samples=100):
    feature_extractor = InceptionV3FeatureExtractor(device)
    
    # Extract features for real images
    real_features = []
    # Convert real_imgs [N, 3, 64, 64] into list of images
    for img in real_imgs:
        real_features.append(feature_extractor.extract_features([img]))
    real_features = np.concatenate(real_features, axis=0)
    
    # Generate fake images and extract features
    fake_features = []
    for i in range(num_samples):
        fake_img = generate_fake_image(model, noise_scheduler, device)
        # Denormalize image if needed (assumes image was generated in same [-1,1] range)
        fake_img = (fake_img * 0.5 + 0.5).clamp(0,1)
        fake_features.append(feature_extractor.extract_features([fake_img]))
    fake_features = np.concatenate(fake_features, axis=0)
    
    fid_score = calculate_fid(real_features, fake_features)
    return fid_score

# Prepare real images from your dataloader (using the same dataloader from earlier)
def get_all_real_images(dataloader, max_imgs=1000):
    all_images = []
    for _, (imgs, _) in enumerate(dataloader):
        all_images.append(imgs)
        if len(torch.cat(all_images, dim=0)) >= max_imgs:
            break
    return torch.cat(all_images, dim=0)[:max_imgs]

import numpy as np

print("Collecting real images for FID evaluation...")
real_imgs = get_all_real_images(dataloader)

print("Computing FID score for diffusion model...")
fid_score = compute_fid(real_imgs, model, noise_scheduler, device, num_samples=min(100, len(real_imgs)))

print(f"Fréchet Inception Distance (FID): {fid_score:.4f}")